In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

# 웹 드라이버 초기화
driver = webdriver.Chrome()

# 사이트 접속
driver.get("https://www.renault.co.kr/ko/service/svc_maintenance.jsp")

# 지역 목록
regions = ["서울특별시", "부산광역시", "대구광역시", "광주광역시", "인천광역시", "대전광역시", 
           "울산광역시", "세종특별자치시", "제주특별자치도", "강원도", "경기도", "경상남도", 
           "경상북도", "전라남도", "전라북도", "충청남도", "충청북도"]

# 전체 데이터를 저장할 리스트
all_data = []

# 각 지역별로 데이터 수집
for region in regions:
    wait = WebDriverWait(driver, 10)
    select_element = wait.until(EC.presence_of_element_located((By.ID, "netsido")))
    select = Select(select_element)
    select.select_by_visible_text(region)
    time.sleep(1)  # 페이지 로딩 대기

    store_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#store_list .store .sbtn_wrap .btn_primary")))

    for button in store_list:
        button.click()  # 상세보기 버튼 클릭
        time.sleep(1)  # 페이지 로딩 대기

        # 수집할 정보 딕셔너리 생성
        store_info = {"지역": region}

        # 네트워크
        network_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '네트워크')]/following-sibling::dd")
        store_info["업체명"] = network_element.text if network_element else None

        # 전화번호
        phone_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '전화번호')]/following-sibling::dd")
        store_info["전화번호"] = phone_element.text if phone_element else None

        # 주소
        address_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '주소')]/following-sibling::dd")
        store_info["주소"] = address_element.text if address_element else None

        # 전기차 정비레벨
        electric_level_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '전기차 정비레벨')]/following-sibling::dd")
        store_info["전기차 정비레벨"] = electric_level_element.text if electric_level_element else None

        # 상용차 정비
        commercial_repair_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '상용차 정비')]/following-sibling::dd")
        store_info["상용차 정비"] = commercial_repair_element.text if commercial_repair_element else None

        # 영업시간
        hours_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '영업시간')]/following-sibling::dd")
        store_info["영업시간"] = hours_element.text if hours_element else None

        # 케어서비스 24/7 (주말 및 야간입고)
        care_service_element = driver.find_element(By.XPATH, "//div[@id='map-info']//dt[contains(text(), '케어서비스 24/7')]/following-sibling::dd")
        store_info["케어서비스 24/7 (주말 및 야간입고)"] = care_service_element.text if care_service_element else None

        # 수집된 데이터를 리스트에 추가
        all_data.append(store_info)

# 모든 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_data)

df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시', '', regex=True)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

df

df.to_excel('르노코리아_최종.xlsx')